In [22]:
# %%
import os
from datetime import datetime
from utils.data_lightning import preloading
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mat
import matplotlib.patches as patches

import torch as th
from torch.autograd import Variable
import torch.nn as nn

from models.ae import seq2seq_ConvLSTM
import models

import argparse
import pytorch_ssim
import time

mat.use("Agg") # headless mode
#mat.rcParams['text.color'] = 'w'


In [27]:
dataset = preloading.SWEDataModule(
    root="../datasets/arda/256/",
    test_size=0.1,
    val_size=0,
    past_frames=4,
    future_frames=1,
    partial=0.01,
    filtering=True,
    batch_size=4,
    workers=4,
    image_size=192,
    shuffle=False,
    dynamicity=2e-1,
    caching=False,
    downsampling=True
)

dataset.prepare_data()

[x] 1 areas found
Area 0 - sequences: 59
- - - - - - - - - - - - x x x x x - - - - 

In [7]:
if th.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"

device = th.device(dev)

# -------------- Model
print("[x] Loading model weights")

net = seq2seq_ConvLSTM.EncoderDecoderConvLSTM(nf=32, in_chan=4, out_chan=3)

multigpu = False
if multigpu:
    net = nn.DataParallel(net)

weights = "../trained_models/train_135_02_09_2021_00_23_36/model.weights"
net.load_state_dict(
    th.load(weights, map_location=device)
)

net = net.to(device)
net.eval() # evaluation mode

print("[!] Successfully loaded weights from {}".format(weights))

[x] Loading model weights
[!] Successfully loaded weights from ../trained_models/train_135_02_09_2021_00_23_36/model.weights


In [20]:
no_of_layers=0
conv_layers=[]
 
model_children=list(net.children())

for child in model_children:
  if type(child)==models.ae.ConvLSTMCell.ConvLSTMCell:
    no_of_layers+=1
    conv_layers.append(child.conv)

In [ ]:
results = [conv_layers[0](img)]
for i in range(1, len(conv_layers)):
    results.append(conv_layers[i](results[-1]))
outputs = results